# Análise de Subtipos Moleculares

Este notebook integra todas as etapas do fluxo de trabalho para análise de subtipos moleculares, utilizando os módulos modularizados.

### **Configuração Inicial e Importações**

In [ ]:
# Importações padrão
import pandas as pd
import importlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import shap

# Importar módulos personalizados
from modules.data_loader import DataLoader
from modules.preprocessor import Preprocessor
from modules.modeling import RandomForestKFoldRunner
from modules.visualization import Visualization

from modules.panel_manager import PanelManager
import modules.model_evaluation as model_evaluation_module
import modules.visualization as visualization_module
from modules.model_evaluation import ModelEvaluator

In [ ]:
# Configurar diretórios
data_dir = 'datas'
output_dir = 'outputs'
results_dir = 'results'

# Criar diretórios se não existirem
for directory in [output_dir, results_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# **Etapa 1. Geração da Tabela de Genes Vs Subtipos**

### 1.1. Carregamento de Dados

In [ ]:
# Inicializar o carregador de dados
data_loader = DataLoader(data_dir=data_dir)

# Carregar dados clínicos
clinical_data = data_loader.load_clinical_data()
clinical_data.head()

### Informações sobre o conjunto de dados

**Subtipos Moleculares**

In [ ]:
# Estatísticas dos subtipos
subtype_stats = data_loader.get_subtype_statistics(clinical_data)

**SNV**

In [ ]:
# Carregar dados somáticos
somatic_data = data_loader.load_somatic_data()
somatic_data.head()

In [ ]:
print(f"Quanditade total de amostras: {len(somatic_data['Sample_ID'])}")
print(f"Quantidade de casos diferentes: {len(somatic_data['Sample_ID'].unique())}")

### 1.2. Pré-processamento de Dados

In [ ]:
# Inicializar o pré-processador
preprocessor = Preprocessor(output_dir=output_dir)

# Mesclar dados clínicos e somáticos
merged_df = preprocessor.merge_clinical_and_somatic(clinical_data, somatic_data)
merged_df.head()

### 1.3 - Fazendo operações na tabela mesclada

As amostras que possuem na coluna `effect` o valor `synonymous_variant` são irrelevantes para esse estudo. Por isso serão exluida todas as amostras com esse valor. 

In [ ]:
# Criar matriz de genes
gene_matrix = preprocessor.create_gene_matrix(somatic_data, merged_df)
print(f"Dimensões da matriz de genes: {gene_matrix.shape}")
gene_matrix.head()

# **Etapa 2. K-Fold e Geração dos valores SHAP**

Uma vez gereda a tabela dos genes e subtipos moleculares, executa o k-fold para obter os valores SHAP de cada gene.

### 2.1. Visualização Exploratória

In [ ]:
# Inicializar o visualizador
viz = Visualization(output_dir=output_dir)

In [ ]:
# Plotar distribuição de subtipos
fig_dist = viz.plot_subtype_distribution(clinical_data)
viz.save_figure(fig_dist, 'subtype_distribution')
fig_dist

In [ ]:
# Plotar frequência de genes
df_final = pd.read_csv('outputs/gene_subtype_table.csv')
fig_freq = viz.plot_gene_frequency(df_final, top_n=20)
viz.save_figure(fig_freq, 'gene_frequency_top20')
fig_freq


In [ ]:
# Plotar heatmap de genes por subtipo
fig_heatmap = viz.plot_gene_subtype_heatmap(df_final, top_n=15)
viz.save_figure(fig_heatmap, 'gene_subtype_heatmap')
fig_heatmap


### 2.2. Preparação para Modelagem

##### Separando grupo teste hold out
`Verificando se a lista dos casos de test utilizados como teste para o ensemble estão no conjunto dos casos restantes`

In [ ]:
test_list = ['TCGA-D7-A6EY', 'TCGA-CD-A48C', 'TCGA-D7-5578', 'TCGA-CG-4469', 'TCGA-HU-A4GD', 'TCGA-BR-8371', 'TCGA-CD-A4MJ', 'TCGA-BR-8291', 'TCGA-RD-A8N4', 'TCGA-BR-6801', 'TCGA-CG-4462', 'TCGA-RD-A8N2', 'TCGA-D7-6822', 'TCGA-CD-8530', 'TCGA-D7-A748', 'TCGA-HU-A4H3', 'TCGA-HF-A5NB', 'TCGA-BR-7715', 'TCGA-D7-A6EZ', 'TCGA-B7-A5TN', 'TCGA-HF-7136', 'TCGA-CG-4444', 'TCGA-D7-6518', 'TCGA-BR-7723', 'TCGA-RD-A7BS', 'TCGA-D7-6521', 'TCGA-BR-6707', 'TCGA-BR-A4IZ', 'TCGA-D7-A747', 'TCGA-VQ-A8PD', 'TCGA-VQ-A923', 'TCGA-D7-A6F0', 'TCGA-HU-A4H8', 'TCGA-VQ-A8PF', 'TCGA-HU-A4GF', 'TCGA-BR-7196', 'TCGA-BR-6852', 'TCGA-BR-7959', 'TCGA-VQ-A8P8', 'TCGA-BR-7957', 'TCGA-BR-8590', 'TCGA-MX-A5UJ', 'TCGA-BR-8679', 'TCGA-BR-6566', 'TCGA-D7-8572', 'TCGA-VQ-AA69', 'TCGA-HF-7132', 'TCGA-CG-4443', 'TCGA-VQ-A8PB', 'TCGA-BR-4279', 'TCGA-CD-8535', 'TCGA-CD-A486', 'TCGA-VQ-A8PX', 'TCGA-HU-8604', 'TCGA-MX-A5UG', 'TCGA-BR-4357', 'TCGA-HU-A4G9', 'TCGA-BR-A4QI', 'TCGA-HU-A4GU', 'TCGA-BR-A452', 'TCGA-D7-6519', 'TCGA-D7-8570', 'TCGA-HU-A4H6', 'TCGA-CG-5720', 'TCGA-BR-7901', 'TCGA-BR-6706', 'TCGA-BR-8687', 'TCGA-VQ-A928', 'TCGA-BR-8081', 'TCGA-3M-AB47', 'TCGA-CG-4436', 'TCGA-BR-8058', 'TCGA-VQ-A91W', 'TCGA-BR-4361', 'TCGA-BR-4370', 'TCGA-D7-A6EX', 'TCGA-BR-7707', 'TCGA-CG-4477', 'TCGA-CG-5726', 'TCGA-BR-4253', 'TCGA-D7-A6F2', 'TCGA-HU-8249']

In [ ]:
# Dividir em treino/validação e teste
X_train_val, y_train_val, X_test, y_test, sample_ids, train_val_df, test_df = preprocessor.create_train_test_split(test_list, df_final)


### 2.3. **Modelagem com Random Forest** - Treinando Modelos e Obtendo Valores SHAP

In [ ]:
# Treinar e avaliar modelo com K-Fold CV
runner = RandomForestKFoldRunner(
    n_splits=10,
    random_state=42,
    output_dir="outputs",
    n_iter_search=10
)

results = runner.run(X_train_val, y_train_val, train_val_df)


# **Etapa 3 - Gerando Métricas de Avaliação dos Modelos**

In [ ]:
# Inicializa gerenciadores
panel_manager = PanelManager()
evaluator = ModelEvaluator(output_dir=output_dir)
viz = Visualization(output_dir=output_dir)

panel_json_path = os.path.join('datas', 'genetic_panels.json')
# Carrega painéis genéticos (usa caminho padrão datas/genetic_panels.json)
panels = panel_manager.load_panels_from_json(panel_json_path)

# Seleciona painéis para avaliação (todos por padrão)
selected_panels = list(panels.keys())

# Verifica se df_final e test_list estão definidos
assert 'df_final' in globals(), "df_final não encontrado. Garanta que a tabela de genes vs subtipos foi carregada anteriormente."
assert 'test_list' in globals(), "test_list não encontrado. Garanta que a lista de casos de teste foi definida anteriormente."

results_por_painel = {}
for panel_name in selected_panels:
    print(f"=== Avaliando painel: {panel_name} ===")
    panel_out_dir = os.path.join(output_dir, 'panels_results', panel_name)
    os.makedirs(panel_out_dir, exist_ok=True)

    # Executa a geração de métricas alinhada ao notebook 'etapa3-gerando-metricas'
    result = evaluator.metrics_from_json(df_final, panels, panel_name, path=panel_out_dir, test_list=test_list)

    # Calcula relatório médio e plota visualizações
    avg_report = evaluator.average_classification_report(result['reports'])
    fig1 = viz.plot_avg_classification_report_heatmap(avg_report, title=f"Métricas médias por classe - {panel_name}")
    viz.save_figure(fig1, filename='avg_report_heatmap', path=os.path.join(panel_out_dir, 'figures'))
    fig1
    
    fig2 = viz.plot_avg_metrics_bar(avg_report, include_accuracy=True, title=f"Métricas médias (Macro/Weighted) - {panel_name}")
    viz.save_figure(fig2, filename='avg_metrics_bar', path=os.path.join(panel_out_dir, 'figures'))
    fig2
    
    results_por_painel[panel_name] = {
        'avg_report': avg_report,
        'metrics_df': result.get('metrics_df')
    }

print("\n✅ Etapa 3 (modular) finalizada: métricas e visualizações salvas em 'outputs/panels_results/<painel>/figures'")